In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

### Importing the product reviews data.

In [23]:
product_reviews = pd.read_csv('reviews.csv')

In [24]:
product_reviews.head()

,asin,name,date,rating,review
0,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-09-06,1,I bought this hair oil after viewing so many g...
1,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-08-14,5,Used This Mama Earth Newly Launched Onion Oil ...
2,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-10-19,1,So bad product...My hair falling increase too ...
3,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-09-16,1,Product just smells similar to navarathna hair...
4,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-08-18,5,I have been trying different onion oil for my ...


In [25]:
product_reviews.shape

(2782, 5)

In [41]:
product_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2778 entries, 0 to 2781
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   asin       2778 non-null   object
 1   name       2778 non-null   object
 2   date       2778 non-null   object
 3   rating     2778 non-null   int64 
 4   review     2778 non-null   object
 5   brandName  2778 non-null   object
dtypes: int64(1), object(5)
memory usage: 151.9+ KB


# Missing Values

In [26]:
missing = pd.concat([product_reviews.isnull().sum(), 100 * product_reviews.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by=['count', '%'],ascending=True)

,count,%
asin,0,0.000000
name,0,0.000000
date,0,0.000000
rating,0,0.000000
review,4,0.143781


In [28]:
product_reviews.dropna(inplace=True)

In [29]:
product_reviews.shape

(2778, 5)

#### List of Unique Products

In [27]:
product_reviews['name'].nunique()

122

That is, we have a total of 122 unique products in the data set provided.

Number of reviews for each product 

In [18]:
product_reviews['name'].value_counts().head()

Tata-Tea-Gold-500g                        60
Society-Tea-Masala-Jar-250g               40
Himalaya-4003F-Baby-Lotion-400ml          40
Tata-Tea-Premium-North-1kg                40
Dettol-Disinfectant-Cleaner-Home-Fresh    40
Name: name, dtype: int64

### Now we will try to find the unique brands that these products belong to and the number of reviews each brand has. We can do this as we can see a pattern in the name and extracting the first word would give us the brand associated with the product.

In [31]:
product_reviews['brandName'] = product_reviews['name'].str.split('-').str[0]
product_reviews.head()

,asin,name,date,rating,review,brandName
0,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-09-06,1,I bought this hair oil after viewing so many g...,Mamaearth
1,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-08-14,5,Used This Mama Earth Newly Launched Onion Oil ...,Mamaearth
2,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-10-19,1,So bad product...My hair falling increase too ...,Mamaearth
3,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-09-16,1,Product just smells similar to navarathna hair...,Mamaearth
4,B07W7CTLD1,Mamaearth-Onion-Growth-Control-Redensyl,2019-08-18,5,I have been trying different onion oil for my ...,Mamaearth


In [35]:
product_reviews['brandName'].unique()

array(['Mamaearth', 'Godrej', 'Titan', 'Maaza', 'Paper', 'Indiana',
       'Coca', 'Natural', 'Maggi', 'Glucon', 'Amul', 'Patanjali',
       'PATANJALI', 'Dettol', 'Savlon', 'Cinthol', 'Britannia',
       'NutriChoice', 'Streax', 'Himalaya', 'Society', 'Tata', 'Fastrack',
       'Reflex', 'MYSORE', 'Mysore'], dtype=object)

In [37]:
product_reviews['brandName'] = product_reviews['brandName'].str.replace('PATANJALI', 'Patanjali')
product_reviews['brandName'] = product_reviews['brandName'].str.replace('MYSORE', 'Mysore')

In [38]:
product_reviews['brandName'].nunique()

24

#### There are a total of 24 unique brands that have their product reviews in the dataset

In [40]:
product_reviews['brandName'].value_counts().head()

Godrej       200
Cinthol      200
Mamaearth    200
Titan        200
Himalaya     200
Name: brandName, dtype: int64